In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import sqrt, pi, arctan2, cos, sin, stats
from scipy.ndimage import uniform_filter

original_frame1 = cv2.imread("data/tennis492.jpg")
original_frame2 = cv2.imread("data/tennis493.jpg")

# convert to gray-scale
frame1 = cv2.cvtColor(original_frame1,cv2.COLOR_BGR2GRAY)
frame2 = cv2.cvtColor(original_frame2,cv2.COLOR_BGR2GRAY)

frame1 = cv2.resize(frame1, (256,256))
frame2 = cv2.resize(frame2, (256,256))

# Gets the optical flow [<dx,dy>] from two frames
def getOpticalFlow(imPrev, imNew):
    flow = cv2.calcOpticalFlowFarneback(imPrev, imNew, flow=None, pyr_scale=.5, levels=3, winsize=9, iterations=1, poly_n=3, poly_sigma=1.1, flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    return flow

flow = getOpticalFlow(frame1, frame2)

In [4]:
# Compute the Histogram of Optical Flow (HoF) from the given optica flow

def hof(flow, orientations=9, pixels_per_cell=(10, 10),
        cells_per_block=(3, 3), visualise=False, normalise=False, motion_threshold=1.):
    flow = np.atleast_2d(flow)
    if flow.ndim < 3:
        raise ValueError("Requires dense flow in both directions")

    if normalise:
        flow = sqrt(flow)

    if flow.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        flow = flow.astype('float')

    gx = np.zeros(flow.shape[:2])
    gy = np.zeros(flow.shape[:2])
    # gx[:, :-1] = np.diff(flow[:,:,1], n=1, axis=1)
    # gy[:-1, :] = np.diff(flow[:,:,0], n=1, axis=0)

    gx = flow[:,:,1]
    gy = flow[:,:,0]
    
    magnitude = sqrt(gx**2 + gy**2)
    orientation = arctan2(gy, gx) * (180 / pi) % 180

    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    bx, by = cells_per_block

    n_cellsx = int(np.floor(sx // cx))  # number of cells in x
    n_cellsy = int(np.floor(sy // cy))  # number of cells in y

    # compute orientations integral images
    orientation_histogram = np.zeros((n_cellsy, n_cellsx, orientations))
    subsample = np.index_exp[cy / 2:cy * n_cellsy:cy, cx / 2:cx * n_cellsx:cx]
    for i in range(orientations-1):
        #create new integral image for this orientation
        # isolate orientations in this range

        temp_ori = np.where(orientation < 180 / orientations * (i + 1),
                            orientation, -1)
        temp_ori = np.where(orientation >= 180 / orientations * i,
                            temp_ori, -1)
        # select magnitudes for those orientations
        cond2 = (temp_ori > -1) * (magnitude > motion_threshold)
        temp_mag = np.where(cond2, magnitude, 0)

        temp_filt = uniform_filter(temp_mag, size=(cy, cx))
        orientation_histogram[:, :, i] = temp_filt[subsample]

    ''' Calculate the no-motion bin '''
    temp_mag = np.where(magnitude <= motion_threshold, magnitude, 0)

    temp_filt = uniform_filter(temp_mag, size=(cy, cx))
    orientation_histogram[:, :, -1] = temp_filt[subsample]

    # now for each cell, compute the histogram
    hof_image = None

    if visualise:
        from skimage import draw

        radius = min(cx, cy) // 2 - 1
        hof_image = np.zeros((sy, sx), dtype=float)
        for x in range(n_cellsx):
            for y in range(n_cellsy):
                for o in range(orientations-1):
                    centre = tuple([y * cy + cy // 2, x * cx + cx // 2])
                    dx = int(radius * cos(float(o) / orientations * np.pi))
                    dy = int(radius * sin(float(o) / orientations * np.pi))
                    rr, cc = draw.bresenham(centre[0] - dy, centre[1] - dx,
                                            centre[0] + dy, centre[1] + dx)
                    hof_image[rr, cc] += orientation_histogram[y, x, o]

    """
    The fourth stage computes normalisation, which takes local groups of
    cells and contrast normalises their overall responses before passing
    to next stage. Normalisation introduces better invariance to illumination,
    shadowing, and edge contrast. It is performed by accumulating a measure
    of local histogram "energy" over local groups of cells that we call
    "blocks". The result is used to normalise each cell in the block.
    Typically each individual cell is shared between several blocks, but
    its normalisations are block dependent and thus different. The cell
    thus appears several times in the final output vector with different
    normalisations. This may seem redundant but it improves the performance.
    We refer to the normalised block descriptors as Histogram of Oriented
    Gradient (hog) descriptors.
    """

    n_blocksx = (n_cellsx - bx) + 1
    n_blocksy = (n_cellsy - by) + 1
    normalised_blocks = np.zeros((n_blocksy, n_blocksx,
                                  by, bx, orientations))

    for x in range(n_blocksx):
        for y in range(n_blocksy):
            block = orientation_histogram[y:y+by, x:x+bx, :]
            eps = 1e-5
            normalised_blocks[y, x, :] = block / sqrt(block.sum()**2 + eps)

    """
    The final step collects the hof descriptors from all blocks of a dense
    overlapping grid of blocks covering the detection window into a combined
    feature vector for use in the window classifier.
    """

    if visualise:
        return normalised_blocks.ravel(), hof_image
    else:
        return normalised_blocks.ravel()

In [28]:
hof_result = hof(flow)
print hof_result
print hof_result.shape

[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,  -2.66055072e-19
  -1.96920303e-34   1.25529637e-03]
(42849,)


In [20]:
from scipy.stats import binned_statistic
def hofom(flow, orientations=4, magnitudes=4, pixels_per_cell=(8,8)):
    gx = flow[:,:,1]
    gy = flow[:,:,0]
    
    magnitude = sqrt(gx**2 + gy**2)
    magnitude_ravel = magnitude.ravel()
    magnitude_ranked = stats.rankdata(magnitude_ravel, "average")/float(len(magnitude_ravel))
    orientation = arctan2(gy, gx) * (180 / pi) % 180
    orientation_ravel = orientation.ravel()
    orientation_ranked = stats.rankdata(orientation_ravel, "average")/float(len(orientation_ravel))
    
    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    ncx = sx//cx
    ncy = sy//cy
    
    ret = np.zeros((ncx, ncy, orientations*magnitudes))
    
    #(256,256) -> (32,32,16)
    for i in range(sy):
        for j in range(sx):
            # map ori to {orientations} values
            pori = orientation_ranked[i*sx+j]
            iori = int(pori * orientations + 1)
            # map mag to {magnitudes} values
            pmag = magnitude_ranked[i*sx+j]
            imag = int(pmag * magnitudes + 1)
            iy = i%ncy
            ix = j%ncx
            quantile = iori*imag-1
            if quantile > orientations*magnitudes:
                quantile = orientations*magnitudes - 1
            ret[ix, iy, quantile] += 1
    return ret.ravel()

In [13]:
mag_ranked = stats.rankdata([1,23,4,5], "min")/4.
mag_ranked

array([ 0.25,  1.  ,  0.5 ,  0.75])

In [21]:
result = hofom(flow)

In [23]:
result.shape

(16384,)

In [24]:
import cv2, os
import numpy as np
from matplotlib import pyplot as plt

In [39]:
# Adding a modifier to reduce number of dataset
MODIFIER=1

# Collect the file path of all the running and walking videos, we will only be using these 2 classes
RUN_DIR = "hof/run/"
RUN_FILES = os.listdir(RUN_DIR)
RUN_FILES = [RUN_DIR + f for f in RUN_FILES]
WALK_DIR = "hof/walk/"
WALK_FILES = os.listdir(WALK_DIR)
WALK_FILES = [WALK_DIR + f for f in WALK_FILES]

# Use equal number of data from each class
nc = min(len(RUN_FILES), len(WALK_FILES)) / MODIFIER
print "nc:", nc
RUN_FILES = RUN_FILES[0:nc]
WALK_FILES = WALK_FILES[0:nc]

RATIO = 0.9
offset = int(np.floor(nc*RATIO))
print "offset:", offset

# Split test and training at a ratio of 1:9
train_files = RUN_FILES[0:offset] + WALK_FILES[0:offset]
test_files = RUN_FILES[offset:nc] + WALK_FILES[offset:nc]

# Put the labels in vectors
train_labels = np.zeros(offset*2, int)
train_labels[0:offset] = 1 #RUN=1
train_labels[offset:offset*2] = 2 #WALK=2

test_len = nc-offset
test_labels = np.zeros(test_len*2, int)
test_labels[0:test_len] = 1 #RUN=1
test_labels[test_len:test_len*2] = 2 #WALK=2

print "train files:", len(train_files)
print "train labels:", len(train_labels)
print "test files:", len(test_files)
print "test labels:", len(test_labels)

nc: 23
offset: 20
train files: 40
train labels: 40
test files: 6
test labels: 6


In [26]:
def getOpticalFlow(imPrev, imNew):
    flow = cv2.calcOpticalFlowFarneback(imPrev, imNew, flow=None, pyr_scale=.5, levels=3, winsize=9, iterations=1, poly_n=3, poly_sigma=1.1, flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    return flow

In [30]:
FIXED_WIDTH = 256
FIXED_HEIGHT = 256
def normalizeFrame(frame_original):
    frame_gray = cv2.cvtColor(frame_original,cv2.COLOR_BGR2GRAY)
    frame_gray_resized = cv2.resize(frame_gray, (FIXED_WIDTH, FIXED_HEIGHT))
    return frame_gray_resized

In [35]:
# get the Histogram of Optical Flow from two images
def getHOFOM(frame1, frame2):
    flow = getOpticalFlow(frame1, frame2)
    return hofom(flow)

In [41]:
# get the Histogram of Optical Flows of a video grouped sequentially in a 1D array
def getSequentialHOFOM(video_path):
    print "Processing", video_path
    hofs = []
    cap = cv2.VideoCapture(video_path)
    ret1, frame1 = cap.read()
    frame1 = normalizeFrame(frame1)
    while(cap.isOpened()):
        ret2, frame2 = cap.read()
        if ret2 == True:
            frame2 = normalizeFrame(frame2)
            hof_array = getHOFOM(frame1, frame2)
            hofs = np.concatenate((hofs, hof_array),axis=0)
#             show2images(frame1, frame2)
            frame1 = frame2
        else:
            break
    return hofs

In [46]:
train = [getSequentialHOFOM(p) for p in train_files]
test = [getSequentialHOFOM(p) for p in test_files]

Processing hof/run/20060723sfjffangelina_run_f_cm_np2_fr_med_3.avi
Processing hof/run/20060723sfjffangelina_run_f_nm_np1_ri_med_1.avi
Processing hof/run/20060723sfjffangelina_run_f_nm_np1_ri_med_2.avi
Processing hof/run/20060723sfjffbartsinger_run_f_nm_np1_fr_med_1.avi
Processing hof/run/20060723sfjffbumblebeesuitman_run_f_cm_np2_ri_med_1.avi
Processing hof/run/20060723sfjffinallseriousness_run_f_cm_np1_fr_med_0.avi
Processing hof/run/20060723sfjffinallseriousness_run_f_cm_np2_ba_med_2.avi
Processing hof/run/20060723sfjffinallseriousness_run_u_cm_np1_fr_med_1.avi
Processing hof/run/20060723sfjffjewcy_run_f_nm_np1_ba_med_3.avi
Processing hof/run/20060723sfjffsomelikeitwarmed_run_f_nm_np1_fr_med_6.avi
Processing hof/run/50_FIRST_DATES_run_f_cm_np1_ba_med_12.avi
Processing hof/run/50_FIRST_DATES_run_f_cm_np1_ri_med_21.avi
Processing hof/run/50_FIRST_DATES_run_f_nm_np1_ba_med_20.avi
Processing hof/run/50_FIRST_DATES_run_f_nm_np1_fr_med_34.avi
Processing hof/run/A_Beautiful_Mind_4_run_f_cm_

In [47]:
print len(test[1])
print len(test[2])
max_width = max(np.array([len(i) for i in train]).max(),np.array([len(i) for i in test]).max())
def numpy_fillna(data, width):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(width) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out
train_pad = numpy_fillna(np.array(train), max_width)
test_pad = numpy_fillna(np.array(test), max_width)

1703936
1261568


In [48]:
%%time
from sklearn import svm, metrics

clf = svm.SVC()
clf.fit(train_pad, train_labels)
predict = clf.predict(test_pad)
print metrics.accuracy_score(predict, test_labels)

0.5
CPU times: user 13.6 s, sys: 5.58 s, total: 19.2 s
Wall time: 21.8 s


In [49]:
%%time
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(train_pad, train_labels)
predict = clf.predict(test_pad)
print metrics.accuracy_score(predict, test_labels)

0.5
CPU times: user 12.3 s, sys: 3.19 s, total: 15.5 s
Wall time: 16.1 s


In [50]:
%%time
from sklearn import ensemble
clf = ensemble.RandomForestClassifier()
clf.fit(train_pad, train_labels)
predict = clf.predict(test_pad)
print metrics.accuracy_score(predict, test_labels)

0.166666666667
CPU times: user 4.5 s, sys: 3.23 s, total: 7.73 s
Wall time: 8.41 s


In [51]:
%%time
from sklearn import linear_model
clf = linear_model.LogisticRegression()
clf.fit(train_pad, train_labels)
predict = clf.predict(test_pad)
print metrics.accuracy_score(predict, test_labels)

0.166666666667
CPU times: user 16.1 s, sys: 3.89 s, total: 20 s
Wall time: 21.1 s
